In [1]:
import numpy as np
import pandas as pd


* `.groupby(by, level, sort, axis, as_index)`
* `.groupby().groups`
* `pd.Grouper()`
* `.groupby()[]`
* `for name, group in df.groupby(): print(name), print(group)`
* `.groupby().get_group()`
* `.groupby().aggregate()`
* `.groupby().agg()`

# 1.数据

In [48]:
df1 = pd.DataFrame([('bird', 'Falconiformes', 389.0),
                    ('bird', 'Psittaciformes', 24.0),
                    ('mammal', 'Carnivora', 80.2),
                    ('mammal', 'Primates', np.nan),
                    ('mammal', 'Carnivora', 58)],
                   index = ['falcon', 'parrot', 'lion', 'monkey', 'leopard'],
                   columns = ('class', 'order', 'max_speed'))
df1

,class,order,max_speed
falcon,bird,Falconiformes,389.0
parrot,bird,Psittaciformes,24.0
lion,mammal,Carnivora,80.2
monkey,mammal,Primates,NaN
leopard,mammal,Carnivora,58.0


In [47]:
grouped = df.groupby(by = 'class', axis = 0)
grouped = df.groupby(by = 'class', axis = 'index')
grouped = df.groupby(by = 'class', axis = 1)
grouped = df.groupby(by = 'class', axis = 'columns')
grouped = df.groupby(by = ['class', 'order'])

In [49]:
df2 = pd.DataFrame({'A': ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'foo'],
                    'B': ['one', 'one', 'two', 'three','two', 'two', 'one', 'three'],
                    'C': np.random.randn(8),
                    'D': np.random.randn(8)})
df2

,A,B,C,D
0,foo,one,-0.162823,1.358844
1,bar,one,-0.996084,-0.540771
2,foo,two,1.563274,0.815959
3,bar,three,-0.850813,0.635951
4,foo,two,1.031306,-0.462583
5,bar,two,0.154302,-0.429476
6,foo,one,-1.469394,0.341579
7,foo,three,-0.035241,0.098178


# 2. `.groupby(by, level, axis, sort)`

### 2.1 group by columns `.groupby(by)`

In [16]:
grouped = df2.groupby(by = 'A')
grouped = df2.groupby(by = ['A', 'B'])

### 2.2 gorup by index `.groupby(level)`

In [51]:
df3 = df2.set_index(['A', 'B'])
grouped = df3.groupby(level = df3.index.names.difference(['B'])).sum()
grouped

,C,D
A,,
bar,-1.692595,-0.334296
foo,0.927122,2.151976


### 2.3 gorup by function `.groupby(by = func())`

In [24]:
def get_letter_type(letter):
    if letter.lower() in 'aeiou':
        return 'vowel'
    else:
        return 'consonant'

print(get_letter_type('A'))
print(get_letter_type('B'))
print(get_letter_type('C'))
print(get_letter_type('D'))

vowel
consonant
consonant
consonant


In [23]:
grouped = df.groupby(get_letter_type, axis = 1)

### 2.4 group by 重复index `groupby().first()`, `.groupby().last()`

In [26]:
lst = [1, 2, 3, 1, 2, 3]
s = pd.Series([1, 2, 3, 10, 20, 30], index = lst)
s

1     1
2     2
3     3
1    10
2    20
3    30
dtype: int64

In [27]:
grouped = s.groupby(level = 0)
grouped.first()

1    1
2    2
3    3
dtype: int64

In [28]:
grouped.last()

1    10
2    20
3    30
dtype: int64

In [29]:
grouped.sum()

1    11
2    22
3    33
dtype: int64

#### 2.5 排序 `.goupby(sort)`

##### `.groupby(by, sort)`

In [52]:
df4 = pd.DataFrame({
    'X': ['B', 'B', 'A', 'A'],
    'Y': [1, 2, 3, 4]
})
df4.groupby(['X']).sum()

,Y
X,
A,7
B,3


In [53]:
df4.groupby('X', sort = False).sum()

,Y
X,
B,3
A,7


##### `.get_group()`

In [56]:
df5 = pd.DataFrame({
    'X': ['A', 'B', 'A', 'B'],
    'Y': [1, 4, 3, 2]
})
df5.groupby('X').get_group('A')

,X,Y
0,A,1
2,A,3


In [57]:
df5.groupby('X').get_group('B')

,X,Y
1,B,4
3,B,2


#### 2.6 groupby 对象的属性`.groups`

In [59]:
df2

,A,B,C,D
0,foo,one,-0.162823,1.358844
1,bar,one,-0.996084,-0.540771
2,foo,two,1.563274,0.815959
3,bar,three,-0.850813,0.635951
4,foo,two,1.031306,-0.462583
5,bar,two,0.154302,-0.429476
6,foo,one,-1.469394,0.341579
7,foo,three,-0.035241,0.098178


In [61]:
df2.groupby('A').groups

{'bar': Int64Index([1, 3, 5], dtype='int64'),
 'foo': Int64Index([0, 2, 4, 6, 7], dtype='int64')}

In [62]:
df2.groupby(get_letter_type, axis = 1).groups

{'consonant': Index(['B', 'C', 'D'], dtype='object'),
 'vowel': Index(['A'], dtype='object')}

In [63]:
grouped = df2.groupby(['A', 'B'])
grouped.groups

{('bar', 'one'): Int64Index([1], dtype='int64'),
 ('bar', 'three'): Int64Index([3], dtype='int64'),
 ('bar', 'two'): Int64Index([5], dtype='int64'),
 ('foo', 'one'): Int64Index([0, 6], dtype='int64'),
 ('foo', 'three'): Int64Index([7], dtype='int64'),
 ('foo', 'two'): Int64Index([2, 4], dtype='int64')}

In [64]:
len(grouped)

6

#### 2.7 GroupBy with MultiIndex

In [74]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['doo', 'doo', 'bee', 'bee', 'bop', 'bop', 'bop', 'bop'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]
index = pd.MultiIndex.from_arrays(arrays, names = ['first', 'second', 'third'])
s = pd.Series(np.random.randn(8), 
              index = index)
s

first  second  third
bar    doo     one      0.192343
               two      1.284699
baz    bee     one     -1.025023
               two     -2.243817
foo    bop     one     -0.802752
               two     -0.323456
qux    bop     one     -0.889732
               two      0.143231
dtype: float64

In [71]:
s.groupby(level = 0).sum()

first
bar   -0.763911
baz   -1.166585
foo    0.998833
qux   -0.578779
dtype: float64

In [70]:
s.groupby(level  = 1).sum()

second
one   -1.139938
two   -0.370503
dtype: float64

In [72]:
s.groupby(level = 'first').sum()

first
bar   -0.763911
baz   -1.166585
foo    0.998833
qux   -0.578779
dtype: float64

In [73]:
s.groupby(level = 'second').sum()

second
one   -1.139938
two   -0.370503
dtype: float64

In [76]:
s.groupby(level = ['first', 'second']).sum()

first  second
bar    doo       1.477043
baz    bee      -3.268839
foo    bop      -1.126208
qux    bop      -0.746501
dtype: float64

In [77]:
s.groupby(['first', 'second']).sum()

first  second
bar    doo       1.477043
baz    bee      -3.268839
foo    bop      -1.126208
qux    bop      -0.746501
dtype: float64

In [79]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]
index = pd.MultiIndex.from_arrays(arrays, names = ['first', 'second'])
s = pd.DataFrame({
    'A': [1, 1, 1, 1, 2, 2, 3, 3],
    'B': np.arange(8)
    }, 
    index = index)
s

A  B
first second      
bar   one     1  0
      two     1  1
baz   one     1  2
      two     1  3
foo   one     2  4
      two     2  5
qux   one     3  6
      two     3  7

In [80]:
s.groupby([pd.Grouper(level = 1), 'A']).sum()

B
second A   
one    1  2
       2  4
       3  6
two    1  4
       2  5
       3  7

In [81]:
s.groupby([pd.Grouper(level = 'second'), 'A']).sum()

B
second A   
one    1  2
       2  4
       3  6
two    1  4
       2  5
       3  7

In [82]:
s.groupby(['second', 'A']).sum()

B
second A   
one    1  2
       2  4
       3  6
two    1  4
       2  5
       3  7

#### 2.7 groupby对象中选择列

In [84]:
df2

,A,B,C,D
0,foo,one,-0.162823,1.358844
1,bar,one,-0.996084,-0.540771
2,foo,two,1.563274,0.815959
3,bar,three,-0.850813,0.635951
4,foo,two,1.031306,-0.462583
5,bar,two,0.154302,-0.429476
6,foo,one,-1.469394,0.341579
7,foo,three,-0.035241,0.098178


In [87]:
grouped = df2.groupby(['A'])
grouped_C = grouped['C']
grouped_D = grouped['D']
print(grouped_C)
print(grouped_D)

In [93]:
print(df2['C'].groupby(df2['A']))
print(df2['D'].groupby(df2['A']))

# 3. 迭代groups

In [98]:
df2

,A,B,C,D
0,foo,one,-0.162823,1.358844
1,bar,one,-0.996084,-0.540771
2,foo,two,1.563274,0.815959
3,bar,three,-0.850813,0.635951
4,foo,two,1.031306,-0.462583
5,bar,two,0.154302,-0.429476
6,foo,one,-1.469394,0.341579
7,foo,three,-0.035241,0.098178


In [96]:
for name, group in df2.groupby('A'):
    print(name)
    print(group)

bar
     A      B         C         D
1  bar    one -0.996084 -0.540771
3  bar  three -0.850813  0.635951
5  bar    two  0.154302 -0.429476
foo
     A      B         C         D
0  foo    one -0.162823  1.358844
2  foo    two  1.563274  0.815959
4  foo    two  1.031306 -0.462583
6  foo    one -1.469394  0.341579
7  foo  three -0.035241  0.098178


In [97]:
for name, group in df2.groupby(['A', 'B']):
    print(name)
    print(group)

('bar', 'one')
     A    B         C         D
1  bar  one -0.996084 -0.540771
('bar', 'three')
     A      B         C         D
3  bar  three -0.850813  0.635951
('bar', 'two')
     A    B         C         D
5  bar  two  0.154302 -0.429476
('foo', 'one')
     A    B         C         D
0  foo  one -0.162823  1.358844
6  foo  one -1.469394  0.341579
('foo', 'three')
     A      B         C         D
7  foo  three -0.035241  0.098178
('foo', 'two')
     A    B         C         D
2  foo  two  1.563274  0.815959
4  foo  two  1.031306 -0.462583


# 4.选择group

In [99]:
df2.groupby(['A']).get_group('bar')

,A,B,C,D
1,bar,one,-0.996084,-0.540771
3,bar,three,-0.850813,0.635951
5,bar,two,0.154302,-0.429476


In [100]:
df2.groupby(['A', 'B']).get_group(('bar', 'one'))

,A,B,C,D
1,bar,one,-0.996084,-0.540771


# 5.聚合

In [103]:
df2

,A,B,C,D
0,foo,one,-0.162823,1.358844
1,bar,one,-0.996084,-0.540771
2,foo,two,1.563274,0.815959
3,bar,three,-0.850813,0.635951
4,foo,two,1.031306,-0.462583
5,bar,two,0.154302,-0.429476
6,foo,one,-1.469394,0.341579
7,foo,three,-0.035241,0.098178


In [105]:
df2.groupby(by = 'A').aggregate(np.sum)

,C,D
A,,
bar,-1.692595,-0.334296
foo,0.927122,2.151976


In [104]:
df2.groupby(by = ['A', 'B']).aggregate(np.sum)

C         D
A   B                        
bar one   -0.996084 -0.540771
    three -0.850813  0.635951
    two    0.154302 -0.429476
foo one   -1.632217  1.700423
    three -0.035241  0.098178
    two    2.594580  0.353376

In [108]:
df2.groupby(by = 'A', as_index = False).aggregate(np.sum)

,A,C,D
0,bar,-1.692595,-0.334296
1,foo,0.927122,2.151976


In [107]:
df2.groupby(by = ['A', 'B'], as_index = False).aggregate(np.sum)

,A,B,C,D
0,bar,one,-0.996084,-0.540771
1,bar,three,-0.850813,0.635951
2,bar,two,0.154302,-0.429476
3,foo,one,-1.632217,1.700423
4,foo,three,-0.035241,0.098178
5,foo,two,2.594580,0.353376


In [114]:
df2.groupby('A').sum().reset_index()

,A,C,D
0,bar,-1.692595,-0.334296
1,foo,0.927122,2.151976


In [115]:
df2.groupby(['A', 'B']).sum().reset_index()

,A,B,C,D
0,bar,one,-0.996084,-0.540771
1,bar,three,-0.850813,0.635951
2,bar,two,0.154302,-0.429476
3,foo,one,-1.632217,1.700423
4,foo,three,-0.035241,0.098178
5,foo,two,2.594580,0.353376
